In [19]:
#Importar librerias
import pandas as pd
from sodapy import Socrata
import numpy as np

In [20]:
#Se cargan los datos de inclusion financiera de la pagina de datos abiertos del gobieron con la api Socrata
# https://www.datos.gov.co/Econom-a-y-Finanzas/Inclusi-n-Financiera/kx2f-xjdq/about_data
client = Socrata("www.datos.gov.co", None, timeout=100)
#Solo se llamaran los datos del 2022 en adelaante
fecha = "fecha_corte >= '2022-01-01T00:00:00.000'"
#Ya que se quiere traer la toda la información y al eliminar el limit del get solo se traen 1000 valores se usa el siguiente codigo 
resultados = []
Inicio = 0
tamaño_batch = 10000 #Tamaño del batch para el limite en ela busqueda de información
#Descargar por tamaos de batch la información y luego descarga la información del siguiente batch
while True:
    results = client.get("kx2f-xjdq", where=fecha, limit=tamaño_batch, offset=Inicio)
    resultados.extend(results)
    Inicio += tamaño_batch
    if len(results) < tamaño_batch: #Si el la cantidad d einformación traida por el .get es menor al tamaño de batch el programa se detiene
        break

In [21]:
#Convertir archivo a un df de pandas
data = pd.DataFrame.from_records(resultados)

In [22]:
#Los datos de las columnas se cargan en tipo Object por lo que se cammbian segun corresponde
columnas_numericas = ['tipo_entidad', 'codigo_entidad', 'unicap', 'renglon',
       '_1_nro_corresponsales_f_sicos', '_2_nro_corresponsales_f_sicos',
       '_3_nro_corresponsales_f_sicos', '_4_nro_corresponsales_f_sicos',
       '_5_nro_dep_sitos', '_6_monto_dep_sitos', '_7_nro_giros_enviados',
       '_8_monto_giros_enviados', '_9_nro_giros_recibidos',
       '_10_monto_giros_recibidos', '_11_nro_pagos_corresponsales',
       '_12_monto_pagos_corresponsales', '_13_nro_retiros_corresponsales',
       '_14_monto_retiros', '_15_nro_transferencias',
       '_16_monto_transferencias', '_17_nro_transacciones_tr',
       '_18_monto_transacciones', '_19_nro_ctas_ahorro_hasta',
       '_20_saldo_ctas_ahorro_hasta', '_21_nro_ctas_ahorro_1smmlv',
       '_22_saldo_ctas_ahorro_1smmlv', '_23_nro_ctas_ahorr_3smmlv',
       '_24_saldo_ctas_ahorr_3smmlv', '_25_nro_ctas_ahorro_activas',
       '_26_saldo_ctas_ahorro_activas', '_27_nro_ctas_ahorro_mujeres',
       '_28_saldo_ctas_ahorro_mujeres', '_29_nro_ctas_ahorro_hombres',
       '_30_saldo_ctas_ahorro_hombres', '_31_nro_ctas_ahorro',
       '_32_saldo_ctas_ahorro', '_41_nro_cr_dito_consumo',
       '_42_monto_cr_dito_consumo', '_43_nro_cr_dito_consumo',
       '_44_monto_cr_dito_consumo', '_45_nro_cr_dito_consumo',
       '_46_monto_cr_dito_consumo', '_47_nro_cred_cons_bajo_monto',
       '_48_monto_cred_cons_bajo', '_49_nro_cred_cons_bajo_monto',
       '_50_monto_cred_cons_bajo', '_51_nro_cred_cons_bajo_monto',
       '_52_monto_cred_cons_bajo', '_53_nro_cr_dito_vivienda',
       '_54_monto_cr_dito_vivienda', '_55_nro_cr_dito_vivienda',
       '_56_monto_cr_dito_vivienda', '_57_nro_cr_dito_vivienda',
       '_58_monto_cr_dito_vivienda', '_59_nro_microcr_dito_hasta',
       '_60_monto_microcr_dito_hasta', '_61_nro_microcr_dito_1smmlv',
       '_62_monto_microcr_dito_1smmlv', '_63_nro_microcr_dito_2smmlv',
       '_64_monto_microcr_dito_2smmlv', '_65_nro_microcr_dito_3smmlv',
       '_66_monto_microcr_dito_3smmlv', '_67_nro_microcr_dito_4smmlv',
       '_68_monto_microcr_dito_4smmlv', '_69_nro_microcr_dito_10smmlv',
       '_70_monto_microcr_dito_10smmlv', '_71_nro_microcr_dito_mujeres',
       '_72_monto_microcr_dito_mujeres', '_73_nro_microcr_dito_hombres',
       '_74_monto_microcr_dito_hombres', '_75_nro_microcr_dito',
       '_76_monto_microcr_dito', '_77_nro_productos_a_nivel',
       '_78_monto_saldo_productos', '_79_nro_corresponsales_f',
       '_80_nro_corresponsales_f', '_81_nro_corresponsales_propios',
       '_82_nro_corresponsales', '_83_nro_corresponsales_m',
       '_84_nro_corresponsales_m', '_85_nro_dep_sitos', '_86_monto_dep_sitos',
       '_87_nro_giros_enviados', '_88_monto_giros_enviados',
       '_89_nro_giros_recibidos', '_90_monto_giros_recibidos',
       '_91_nro_pagos_corresponsales', '_92_monto_pagos_corresponsales',
       '_93_nro_retiros_corresponsales', '_94_monto_retiros',
       '_95_nro_transferencias', '_96_monto_transferencias',
       '_97_nro_transacciones_tr', '_98_monto_transacciones']
data[columnas_numericas] = data[columnas_numericas].apply(pd.to_numeric)
data['fecha_corte'] = pd.to_datetime(data['fecha_corte'])

In [23]:
#Se detecta que el total de cuentas de algunos producto no coincide con los datos por genero por lo que se procede a calcular los totales
#Vumero de cuentas y saldo de cuentas de ahorro
data['_31_nro_ctas_ahorro'] = data['_27_nro_ctas_ahorro_mujeres'] + data['_29_nro_ctas_ahorro_hombres']
data['_32_saldo_ctas_ahorro'] = data['_28_saldo_ctas_ahorro_mujeres'] + data['_30_saldo_ctas_ahorro_hombres']

#Numero de cuentas y monstos de creditos de consumo
data['_45_nro_cr_dito_consumo'] = data['_41_nro_cr_dito_consumo'] + data['_43_nro_cr_dito_consumo']
data['_46_monto_cr_dito_consumo'] = data['_42_monto_cr_dito_consumo'] + data['_44_monto_cr_dito_consumo']

#Numero de cuentas y montos de creditos de viviendas
data['_57_nro_cr_dito_vivienda'] = data['_53_nro_cr_dito_vivienda'] + data['_55_nro_cr_dito_vivienda']
data['_58_monto_cr_dito_vivienda'] = data['_54_monto_cr_dito_vivienda'] + data['_56_monto_cr_dito_vivienda']

#Numero de cuentas y montos de microcreditos
data['_75_nro_microcr_dito'] = data['_71_nro_microcr_dito_mujeres'] + data['_73_nro_microcr_dito_hombres']
data['_76_monto_microcr_dito'] = data['_72_monto_microcr_dito_mujeres'] + data['_74_monto_microcr_dito_hombres']

#Numero de productos nacionales y montos
data['_77_nro_productos_a_nivel'] = data[['_31_nro_ctas_ahorro', '_51_nro_cred_cons_bajo_monto','_45_nro_cr_dito_consumo', '_57_nro_cr_dito_vivienda', '_75_nro_microcr_dito']].sum(axis = 1)
data['_78_monto_saldo_productos'] = data[['_32_saldo_ctas_ahorro', '_52_monto_cred_cons_bajo','_46_monto_cr_dito_consumo', '_58_monto_cr_dito_vivienda', '_76_monto_microcr_dito']].sum(axis = 1)

In [24]:
#Se calcula las columnas con la suma de productos y montos por sexo
data['nro_producto_mujer'] = data[['_27_nro_ctas_ahorro_mujeres', '_41_nro_cr_dito_consumo', '_47_nro_cred_cons_bajo_monto', '_53_nro_cr_dito_vivienda', '_71_nro_microcr_dito_mujeres']].sum(axis = 1)
data['monto_mujer'] = data[['_28_saldo_ctas_ahorro_mujeres', '_42_monto_cr_dito_consumo', '_48_monto_cred_cons_bajo', '_54_monto_cr_dito_vivienda', '_72_monto_microcr_dito_mujeres']].sum(axis = 1)
data['nro_producto_hombre'] = data[['_29_nro_ctas_ahorro_hombres', '_43_nro_cr_dito_consumo', '_49_nro_cred_cons_bajo_monto', '_55_nro_cr_dito_vivienda', '_73_nro_microcr_dito_hombres']].sum(axis = 1)
data['monto_hombre'] = data[['_30_saldo_ctas_ahorro_hombres', '_44_monto_cr_dito_consumo', '_50_monto_cred_cons_bajo', '_56_monto_cr_dito_vivienda', '_74_monto_microcr_dito_hombres']].sum(axis = 1)

C:\Users\Andres\AppData\Local\Temp\ipykernel_8732\1023943956.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['nro_producto_hombre'] = data[['_29_nro_ctas_ahorro_hombres', '_43_nro_cr_dito_consumo', '_49_nro_cred_cons_bajo_monto', '_55_nro_cr_dito_vivienda', '_73_nro_microcr_dito_hombres']].sum(axis = 1)
C:\Users\Andres\AppData\Local\Temp\ipykernel_8732\1023943956.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['monto_hombre'] = data[['_30_saldo_ctas_ahorro_hombres', '_44_monto_cr_dito_consumo', '_50_monto_cred_

In [25]:
#Se seleccionan las columnas de interes
columnas_interes =['nombre_entidad', 'fecha_corte', 'descrip_uc', 'tipo',
       '_3_nro_corresponsales_f_sicos', '_4_nro_corresponsales_f_sicos', '_17_nro_transacciones_tr',
       '_18_monto_transacciones', '_25_nro_ctas_ahorro_activas', '_27_nro_ctas_ahorro_mujeres',
       '_28_saldo_ctas_ahorro_mujeres', '_29_nro_ctas_ahorro_hombres',
       '_30_saldo_ctas_ahorro_hombres', '_31_nro_ctas_ahorro',
       '_32_saldo_ctas_ahorro', '_41_nro_cr_dito_consumo',
       '_42_monto_cr_dito_consumo', '_43_nro_cr_dito_consumo',
       '_44_monto_cr_dito_consumo', '_45_nro_cr_dito_consumo',
       '_46_monto_cr_dito_consumo','_77_nro_productos_a_nivel',
       '_78_monto_saldo_productos'
       ] 

In [26]:
#Se crea un nuevo df con estas columnas
df = data[columnas_interes].copy()

In [27]:
# Lista de columnas con valores categoricos
columnas = ['nombre_entidad', 'descrip_uc',	'tipo']

#Mostrar los valores únicos
for columna in columnas:
    valores_unicos = df[columna].unique()
    print(f"Valores únicos para la columna '{columna}':")
    print(valores_unicos)
    print()

Valores únicos para la columna 'nombre_entidad':
['Citibank' 'Bancamia S.A.' 'Bancolombia' 'Banco W S.A.' 'Banagrario'
 'Banco de Bogotá' 'Lulo Bank' 'Banco de Occidente' 'Banco Pichincha S.A.'
 'Banco Santander' 'Itau' 'BBVA Colombia' 'Banco Popular'
 'Banco Falabella S.A.' 'AV Villas' 'Banco Davivienda'
 'Scotiabank Colpatria S.A.' 'Banco Serfinanza S.A.' 'Mibanco S.A.'
 'Banco Caja Social S.A.' 'Bancoomeva' 'Banco GNB Sudameris' 'Coopcentral'
 'Credifinanciera S.A.' 'Finandina' 'Tuya' 'Corficolombiana S.A.'
 'Giros y Finanzas C.F.' 'Bancar Tecnología C.F.'
 'Financiera Juriscoop C.F.' 'IRIS C.F.' 'Credifamilia' 'Coltefinanciera'
 'Crezcamos' 'RCI Colombia S.A.' 'Confiar' 'Coofinep'
 'JFK Cooperativa Financiera' 'Cotrafa'
 'Cooperativa Financiera de Antioquia' 'Tecnipagos' 'Movii S.A.' 'Pgde'
 'Aval Soluciones Digitales S.A.' 'Coink S.A.' 'Banco Mundo Mujer S.A.'
 'BNP Paribas' 'La Hipotecaria'
 'GM Financial Colombia S.A. Compañía De Financiamiento' 'Global66' 'N/A']

Valores únicos

In [28]:
#Se elimina las filas que tienen todoss sus valores numericos en 0
numeric_cols = df.select_dtypes(include=[np.number])
mask = (numeric_cols != 0).any(axis=1)
df = df[mask]

In [29]:
#Se eliminan las columnas duplicadas manteniendo el primer registro
df = df.drop_duplicates()

In [30]:
#Se eliminan las columnas con el resultado nacional por entidad
df = df.loc[df['descrip_uc'] != 'TOTAL NACIONAL'] 

In [31]:
#Se reinician los indices
df.reset_index(drop=True, inplace=True)

In [32]:
#Se muestra ka informacion de las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362707 entries, 0 to 362706
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   nombre_entidad                 362707 non-null  object        
 1   fecha_corte                    362707 non-null  datetime64[ns]
 2   descrip_uc                     362707 non-null  object        
 3   tipo                           362707 non-null  object        
 4   _3_nro_corresponsales_f_sicos  362707 non-null  float64       
 5   _4_nro_corresponsales_f_sicos  362707 non-null  float64       
 6   _17_nro_transacciones_tr       362707 non-null  float64       
 7   _18_monto_transacciones        362707 non-null  float64       
 8   _25_nro_ctas_ahorro_activas    362707 non-null  float64       
 9   _27_nro_ctas_ahorro_mujeres    362707 non-null  float64       
 10  _28_saldo_ctas_ahorro_mujeres  362707 non-null  float64       
 11  

In [33]:
#Se muestran las principales estadisticas
df.describe()

,fecha_corte,_3_nro_corresponsales_f_sicos,_4_nro_corresponsales_f_sicos,_17_nro_transacciones_tr,_18_monto_transacciones,_25_nro_ctas_ahorro_activas,_27_nro_ctas_ahorro_mujeres,_28_saldo_ctas_ahorro_mujeres,_29_nro_ctas_ahorro_hombres,_30_saldo_ctas_ahorro_hombres,_31_nro_ctas_ahorro,_32_saldo_ctas_ahorro,_41_nro_cr_dito_consumo,_42_monto_cr_dito_consumo,_43_nro_cr_dito_consumo,_44_monto_cr_dito_consumo,_45_nro_cr_dito_consumo,_46_monto_cr_dito_consumo,_77_nro_productos_a_nivel,_78_monto_saldo_productos
count,362707,362707.000000,362707.000000,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05,3.627070e+05
mean,2023-04-05 06:33:03.147830272,11.749732,42.436275,9.024226e+03,3.496576e+09,1.270095e+03,1.386217e+03,1.712122e+09,1.562564e+03,1.907112e+09,2.948782e+03,3.619233e+09,6.515232e+02,5.715143e+08,6.697463e+02,7.501497e+08,1.321270e+03,1.321664e+09,4.293641e+03,5.299282e+09
min,2022-03-31 00:00:00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2022-09-30 00:00:00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2023-03-31 00:00:00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2023-09-30 00:00:00,0.000000,2.000000,8.200000e+01,1.642371e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00,3.104316e+07
max,2024-03-31 00:00:00,21205.000000,105147.000000,4.653116e+07,1.903181e+13,3.507542e+06,2.109060e+06,4.875399e+12,2.150526e+06,5.634338e+12,4.259586e+06,1.050974e+13,7.636646e+06,2.317431e+12,8.477735e+06,2.897149e+12,1.611438e+07,5.211474e+12,1.611438e+07,1.050974e+13
std,NaN,147.176344,827.042415,2.590638e+05,1.093833e+11,2.765978e+04,2.332039e+04,4.523744e+10,2.449974e+04,4.789545e+10,4.770111e+04,9.301039e+10,4.650837e+04,1.579857e+10,4.791371e+04,1.943355e+10,9.431996e+04,3.515044e+10,1.056595e+05,9.968114e+10


In [34]:
#tambien se muestra las columnas tipo object
df.describe(include= ['O'])

,nombre_entidad,descrip_uc,tipo
count,362707,362707,362707
unique,51,34,7
top,Banagrario,ANTIOQUIA,Transacciones y tramites a traves de correspon...
freq,37446,41758,139915


In [35]:
#Convertir el archivo a csv
df.to_csv('df.csv', sep=';', decimal=',', index=False)